<a href="https://colab.research.google.com/github/Kazuya-Y/public/blob/main/Kazuya_%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90_2_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import pandas as pd
df_monthly_test_results=pd.read_csv('/content/monthly_test_results_20260127_210102.csv')
df_study_time_monthly=pd.read_csv('/content/study_time_monthly_20260127_210031.csv')

df_monthly_test_results['テスト種類']
col_needed=['受講生ID','実施日','テスト種類','総合']
df_monthly_test_results=df_monthly_test_results[col_needed]
df_monthly_test_results

,受講生ID,実施日,テスト種類,総合
0,55,2021-01-26,Versant,42
1,55,2021-03-04,Versant,46
2,55,2021-04-07,Versant,44
3,55,2021-05-11,Versant,46
4,55,2021-06-07,Versant,47
...,...,...,...,...
4000,2848,2026-01-11,Versant Speaking & Listening,12
4001,2850,2026-01-20,CASEC,534
4002,2853,2026-01-17,Versant Speaking & Listening,12
4003,2854,2026-01-18,Versant Speaking & Listening,34


In [183]:
vc=df_monthly_test_results['受講生ID'].value_counts()
one_row_values = vc[vc == 1].index
##df_monthly_test_results[df_monthly_test_results['受講生ID'].isin(one_row_values)]
df_monthly_test_results['受講生ID'].nunique()

2024

In [184]:
##1回しかテストを受けていない人を取り除く&同じテストを2回以上受けている人のみを残す
df_monthly_test_results = df_monthly_test_results[df_monthly_test_results.duplicated(subset=["受講生ID", "テスト種類"], keep=False)]
df_monthly_test_results

,受講生ID,実施日,テスト種類,総合
0,55,2021-01-26,Versant,42
1,55,2021-03-04,Versant,46
2,55,2021-04-07,Versant,44
3,55,2021-05-11,Versant,46
4,55,2021-06-07,Versant,47
...,...,...,...,...
3832,2613,2026-01-26,Versant Speaking & Listening,31
3850,2638,2025-10-23,CASEC,373
3851,2638,2026-01-14,CASEC,466
3905,2702,2025-10-28,CASEC,329


In [185]:
df_monthly_test_results = df_monthly_test_results.copy()

# 受講生IDごとに「最も早い実施日」の CEFR_score を取得
first_score = (
    df_monthly_test_results.loc[df_monthly_test_results.groupby("受講生ID")["実施日"].idxmin()]
    .set_index("受講生ID")["総合"]
)

df_monthly_test_results["入会時スコア"] = df_monthly_test_results["受講生ID"].map(first_score)

# 受講生IDごとに「最も遅い実施日」の CEFR_score を取得
last_score = (
    df_monthly_test_results.loc[df_monthly_test_results.groupby("受講生ID")["実施日"].idxmax()]
    .set_index("受講生ID")["総合"]
)

df_monthly_test_results["退会時スコア"] = df_monthly_test_results["受講生ID"].map(last_score)
df_monthly_test_results['スコア変化'] =df_monthly_test_results['退会時スコア']-df_monthly_test_results['入会時スコア']


In [186]:
df_monthly_test_results

,受講生ID,実施日,テスト種類,総合,入会時スコア,退会時スコア,スコア変化
0,55,2021-01-26,Versant,42,42,46,4
1,55,2021-03-04,Versant,46,42,46,4
2,55,2021-04-07,Versant,44,42,46,4
3,55,2021-05-11,Versant,46,42,46,4
4,55,2021-06-07,Versant,47,42,46,4
...,...,...,...,...,...,...,...
3832,2613,2026-01-26,Versant Speaking & Listening,31,23,31,8
3850,2638,2025-10-23,CASEC,373,373,466,93
3851,2638,2026-01-14,CASEC,466,373,466,93
3905,2702,2025-10-28,CASEC,329,329,431,102


In [187]:
##散布図を描くために必要な列、行のみにする

df_monthly_test_results_cal=df_monthly_test_results[['受講生ID','テスト種類','入会時スコア','退会時スコア', 'スコア変化']].drop_duplicates()
df_monthly_test_results_cal['受講生ID'].duplicated().any()
##'受講生ID'に重複なし
##df_monthly_test_results_cal['score_change'].value_counts().sort_index(ascending=False)
df_monthly_test_results_cal

,受講生ID,テスト種類,入会時スコア,退会時スコア,スコア変化
0,55,Versant,42,46,4
13,126,Versant,32,45,13
16,128,Versant,37,47,10
22,131,Versant,44,45,1
24,132,Versant,43,52,9
...,...,...,...,...,...
3824,2605,Versant Speaking & Listening,43,53,10
3827,2608,CASEC,497,519,22
3831,2613,Versant Speaking & Listening,23,31,8
3850,2638,CASEC,373,466,93


In [188]:
##学習時間の集計をする。分単位のデータから時間単位のデータも作成する

df_study_time_monthly_20260127_210031=pd.read_csv('/content/study_time_monthly_20260127_210031.csv')
##df_study_time_monthly_20260127_210031.columns
df_study_time_sum=df_study_time_monthly_20260127_210031.groupby('受講生ID')['学習時間（分）'].sum().reset_index()
##df_study_time_sum.head(50)
##df_study_time_monthly_20260127_210031[df_study_time_monthly_20260127_210031['受講生ID']==112]

import math
import numpy as np
df_study_time_sum['学習時間（時）'] = df_study_time_sum['学習時間（分）']/60
df_study_time_sum['学習時間（時）'] = np.floor(df_study_time_sum['学習時間（時）'] * 100) / 100
df_study_time_sum = df_study_time_sum.drop(columns=["学習時間（分）"])

df_study_time_sum = df_study_time_sum.rename(columns={"学習時間（時）": "総学習時間（時）"})
df_time_score=pd.merge(df_monthly_test_results_cal,df_study_time_sum, on='受講生ID',how='left')
##df_time_score

In [189]:
df_study_time_weekly_20260127_210021 = pd.read_csv('/content/study_time_weekly_20260127_210021.csv')
#df_study_time_weekly_20260127_210021
df_study_time_avg = df_study_time_weekly_20260127_210021.groupby('受講生ID')['学習時間（分）'].mean().reset_index()

df_study_time_avg['学習時間（時）'] = df_study_time_avg['学習時間（分）']/60
df_study_time_avg['学習時間（時）'] = np.floor(df_study_time_avg['学習時間（時）'] * 100) / 100
df_study_time_avg = df_study_time_avg.drop(columns=["学習時間（分）"])

df_study_time_avg = df_study_time_avg.rename(columns={"学習時間（時）": "週次平均学習時間（時）"})
df_time_score = pd.merge(df_time_score,df_study_time_avg, on='受講生ID',how='left')


In [190]:
df_time_score

,受講生ID,テスト種類,入会時スコア,退会時スコア,スコア変化,総学習時間（時）,週次平均学習時間（時）
0,55,Versant,42,46,4,0.83,0.83
1,126,Versant,32,45,13,NaN,NaN
2,128,Versant,37,47,10,333.06,3.54
3,131,Versant,44,45,1,NaN,NaN
4,132,Versant,43,52,9,23.86,1.49
...,...,...,...,...,...,...,...
1072,2605,Versant Speaking & Listening,43,53,10,308.75,5.93
1073,2608,CASEC,497,519,22,158.33,2.63
1074,2613,Versant Speaking & Listening,23,31,8,61.00,1.24
1075,2638,CASEC,373,466,93,232.41,3.18


In [191]:
df_withdrawal_forms_export = pd.read_csv('/content/withdrawal_forms_export_20260127_210102.csv')
col_withdraw_needed =['受講生ID','フォームバージョン','NPS評価','カリキュラム提案','手続きの分かりやすさ',
       'コストパフォーマンス', 'コーチ評価','英語スキル', '納得感', 'アプリ評価',
       '運営事務局評価']
df_withdrawal_forms_export = df_withdrawal_forms_export[col_withdraw_needed]

#df_withdrawal_forms_export.columns
#'英語力の成長'は'英語スキル'に値がある

In [192]:
pd.set_option("display.max_columns", None)
#df_withdrawal_forms_export
df_time_score_withdrawal=pd.merge(df_time_score,df_withdrawal_forms_export, on='受講生ID',how='left')
df_time_score_withdrawal

,受講生ID,テスト種類,入会時スコア,退会時スコア,スコア変化,総学習時間（時）,週次平均学習時間（時）,フォームバージョン,NPS評価,カリキュラム提案,手続きの分かりやすさ,コストパフォーマンス,コーチ評価,英語スキル,納得感,アプリ評価,運営事務局評価
0,55,Versant,42,46,4,0.83,0.83,2.0,7.0,10.0,10.0,7.0,10.0,8.0,8.0,3.0,NaN
1,126,Versant,32,45,13,NaN,NaN,1.0,8.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
2,128,Versant,37,47,10,333.06,3.54,1.0,10.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
3,131,Versant,44,45,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,132,Versant,43,52,9,23.86,1.49,1.0,8.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2605,Versant Speaking & Listening,43,53,10,308.75,5.93,5.0,5.0,7.0,NaN,NaN,8.0,NaN,8.0,7.0,7.0
1092,2608,CASEC,497,519,22,158.33,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,2613,Versant Speaking & Listening,23,31,8,61.00,1.24,5.0,9.0,8.0,NaN,NaN,9.0,NaN,10.0,8.0,8.0
1094,2638,CASEC,373,466,93,232.41,3.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
from google.colab import files
df_time_score_withdrawal.to_csv("test_time_score.csv", index=False, encoding="utf-8-sig")
files.download("test_time_score.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [194]:
df_study_time_weekly_20260127_210021 = df_study_time_weekly_20260127_210021.drop(columns=["受講生名"])
#df_study_time_sum_weekly = df_study_time_weekly_20260127_210021.groupby('受講生ID')['学習時間（分）'].sum().reset_index()

In [195]:
df_study_time_weekly_20260127_210021

,受講生ID,週開始日,教材名,学習時間（分）
0,29,2022-05-16,TOEIC文法でる1000,35
1,29,2022-05-16,キクタンTOEIC L&Rテスト SCORE990,220
2,29,2022-05-16,公式TOEIC Listening & Reading 問題集8,210
3,29,2022-05-23,キクタンTOEIC L&Rテスト SCORE990,160
4,29,2022-11-28,TOEIC L&R TEST 読解特急2（スピード強化編）,90
...,...,...,...,...
176313,2857,2026-01-19,TOEIC L & R TEST 出る単特急 金のフレーズ,95
176314,2857,2026-01-19,英語のハノン 初級,90
176315,2857,2026-01-26,TED（シャドーイング）,70
176316,2857,2026-01-26,TOEIC L & R TEST 出る単特急 金のフレーズ,55


In [196]:
#df_study_time_weekly_20260127_210021
df_study_time_weekly_sum = df_study_time_weekly_20260127_210021.groupby(["受講生ID", "週開始日"], as_index=False)["学習時間（分）"].sum()
#データがある＝学習時間0ではない


In [197]:
df = df_study_time_weekly_sum.copy()
df["週開始日"] = pd.to_datetime(df["週開始日"])
df = df.sort_values(["受講生ID", "週開始日"])

# 前の週との差（週単位）を計算
diff_weeks = df.groupby("受講生ID")["週開始日"].diff().dt.days.div(7)

# 連続でないところ（差が1週でない）で新しい連続グループを作る
streak_group = diff_weeks.ne(1).groupby(df["受講生ID"]).cumsum()

# 各連続グループの長さ（=連続週数）を数えて、その最大値を受講生ごとに取る
max_streak = (
    df.groupby(["受講生ID", streak_group])
      .size()
      .groupby("受講生ID")
      .max()
      .reset_index(name="最大連続学習週数")
)
max_streak
#df_study_time_weekly_sum = df_study_time_weekly_sum.merge(max_streak, on="受講生ID", how="left")
#df_study_time_weekly_sum

,受講生ID,最大連続学習週数
0,29,2
1,47,3
2,55,1
3,69,2
4,88,9
...,...,...
2260,2847,2
2261,2850,2
2262,2853,2
2263,2854,2


In [198]:
df_time_score_withdrawal = pd.merge(df_time_score_withdrawal,max_streak, on='受講生ID',how='left')
df_time_score_withdrawal

,受講生ID,テスト種類,入会時スコア,退会時スコア,スコア変化,総学習時間（時）,週次平均学習時間（時）,フォームバージョン,NPS評価,カリキュラム提案,手続きの分かりやすさ,コストパフォーマンス,コーチ評価,英語スキル,納得感,アプリ評価,運営事務局評価,最大連続学習週数
0,55,Versant,42,46,4,0.83,0.83,2.0,7.0,10.0,10.0,7.0,10.0,8.0,8.0,3.0,NaN,1.0
1,126,Versant,32,45,13,NaN,NaN,1.0,8.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
2,128,Versant,37,47,10,333.06,3.54,1.0,10.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,21.0
3,131,Versant,44,45,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,132,Versant,43,52,9,23.86,1.49,1.0,8.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2605,Versant Speaking & Listening,43,53,10,308.75,5.93,5.0,5.0,7.0,NaN,NaN,8.0,NaN,8.0,7.0,7.0,13.0
1092,2608,CASEC,497,519,22,158.33,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
1093,2613,Versant Speaking & Listening,23,31,8,61.00,1.24,5.0,9.0,8.0,NaN,NaN,9.0,NaN,10.0,8.0,8.0,14.0
1094,2638,CASEC,373,466,93,232.41,3.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0


In [202]:
from google.colab import files
df_time_score_withdrawal.to_csv("test_time_score.csv", index=False, encoding="utf-8-sig")
files.download("test_time_score.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>